In [168]:
import pandas as pd

hn = pd.read_csv("hero-network.csv")
e = pd.read_csv("edges.csv")
nodes = pd.read_csv("nodes.csv")

import re

def cleaning(string):
    #Pattern for backlash in last position
    pattern = "\/$" 
    newS = re.sub(pattern, "", string)

    #pattern for space in last position
    pattern =  " $"
    newS = re.sub(pattern, "", newS)
    
    return newS

#Extract all the names from the edges
vecHeroRight = e.hero.unique()
cleanHeroRight = set(map(cleaning, vecHeroRight))

#Extract all the names of the heroes that might be wrong (col hero1)
vecHeroWrong = hn.hero1.unique()
newVecHeroWrong = set(map(cleaning, vecHeroWrong))

#Extract all the names of the heroes that might be wrong (col hero2)
vecHeroWrong2 = hn.hero2.unique()
newVecHeroWrong2 = set(map(cleaning, vecHeroWrong2))

#Merge 
mergeHeroWrong = newVecHeroWrong.union(newVecHeroWrong2)

#TODO: convert into function

dic = {}
#Iterate over the wrong hero names
for x in mergeHeroWrong:
    
    #Iterate over the right hero names
    for y in cleanHeroRight:
        dic[x] = x
        if x in y:
            dic[x] = y
            break
            
#Clean the values in both columns
hn.hero1 = hn.hero1.apply(lambda row: cleaning(row))
hn.hero2 = hn.hero2.apply(lambda row: cleaning(row))

#Replace the wrong names with the right ones
hn["hero1"] = hn.hero1.apply(lambda row: dic[row])
hn["hero2"] = hn.hero2.apply(lambda row: dic[row])

#Clean the values in nodes.node
nodes.node =  nodes.node.apply(lambda row: cleaning(row))

#remove duplicates rows from hero network
hn_final = hn[hn.hero1 != hn.hero2]

## First Graph

In [169]:
hn_final = hn_final.sort_values(by = ['hero1', 'hero2']).reset_index(drop = True)

In [170]:
mask = hn_final['hero1'] > hn_final['hero2']

# swap the values in those rows
hn_final.loc[mask, ['hero1', 'hero2']] = hn_final.loc[mask, ['hero2', 'hero1']].values

In [171]:
hn_temp = hn_final
hn_final['Number'] = hn_final.groupby(['hero1', 'hero2']).cumcount().add(1)


In [172]:
hn_final = hn_final.sort_values(by = 'Number', ascending = False)

In [173]:
hn_final = hn_final.drop_duplicates(subset = ['hero1', 'hero2'], keep = 'first')

In [174]:
hn_final = hn_final.groupby(['hero1', 'hero2'], sort = False).agg({'hero1' : 'first', 'hero2' : 'first', 'Number': 'sum'}).reset_index(drop = True)

In [175]:
hn_final['Number'] = hn_final['Number'].apply(lambda x: 1/x)

In [176]:
hn_final = pd.merge(hn_temp, hn_final, on=['hero1','hero2'])

In [177]:
hn_final.drop('Number_x', axis=1, inplace= True)

In [178]:
hn_final.rename(columns={"Number_y":"weight"}, inplace=True)
hn_final.sort_values(by="weight")

,hero1,hero2,weight
415503,MISS AMERICA/MADELIN,PATRIOT/JEFF MACE,0.000528
415314,MISS AMERICA/MADELIN,PATRIOT/JEFF MACE,0.000528
415313,MISS AMERICA/MADELIN,PATRIOT/JEFF MACE,0.000528
415312,MISS AMERICA/MADELIN,PATRIOT/JEFF MACE,0.000528
415311,MISS AMERICA/MADELIN,PATRIOT/JEFF MACE,0.000528
...,...,...,...
384695,COLDHEART,"MACENDALE, KAREN",1.000000
384694,HOBGOBLIN V/JASON PH,"MACENDALE, JASON JR.",1.000000
384693,GRIM HUNTER/KRAVINOF,"MACENDALE, JASON JR.",1.000000
384680,MAA-GOR,PROFESSOR X/CHARLES,1.000000


Il seguente è il grafo con il collegamento tra gli eroi con il relativo peso dato dall'inverso del numero di volte che hanno collaborato

In [186]:
import networkx as nx

In [195]:
a = hn_final.to_numpy()

G = nx.MultiGraph()

In [196]:
G.add_weighted_edges_from(a)

In [198]:
#uncomment to try
#G.edges("MISS AMERICA/MADELIN")

In [199]:
nx.get_edge_attributes(G, 'weight')

{('24-HOUR MAN/EMMANUEL', 'FROST, CARMILLA', 0): 1.0,
 ('24-HOUR MAN/EMMANUEL', 'KILLRAVEN/JONATHAN R', 0): 1.0,
 ('24-HOUR MAN/EMMANUEL', "M'SHULLA", 0): 1.0,
 ('24-HOUR MAN/EMMANUEL', "G'RATH", 0): 1.0,
 ('24-HOUR MAN/EMMANUEL', 'OLD SKULL', 0): 1.0,
 ('FROST, CARMILLA', 'ABRAXAS', 0): 0.3333333333333333,
 ('FROST, CARMILLA', 'ABRAXAS', 1): 0.3333333333333333,
 ('FROST, CARMILLA', 'ABRAXAS', 2): 0.3333333333333333,
 ('FROST, CARMILLA', 'ADAM 3,031', 0): 0.25,
 ('FROST, CARMILLA', 'ADAM 3,031', 1): 0.25,
 ('FROST, CARMILLA', 'ADAM 3,031', 2): 0.25,
 ('FROST, CARMILLA', 'ADAM 3,031', 3): 0.25,
 ('FROST, CARMILLA', 'ATALON', 0): 0.25,
 ('FROST, CARMILLA', 'ATALON', 1): 0.25,
 ('FROST, CARMILLA', 'ATALON', 2): 0.25,
 ('FROST, CARMILLA', 'ATALON', 3): 0.25,
 ('FROST, CARMILLA', 'DEVOURER', 0): 1.0,
 ('FROST, CARMILLA', 'EVE 3,031', 0): 0.25,
 ('FROST, CARMILLA', 'EVE 3,031', 1): 0.25,
 ('FROST, CARMILLA', 'EVE 3,031', 2): 0.25,
 ('FROST, CARMILLA', 'EVE 3,031', 3): 0.25,
 ('FROST, CARMILL

## Second Graph

In [203]:
e.head()

,hero,comic
0,24-HOUR MAN/EMMANUEL,AA2 35
1,3-D MAN/CHARLES CHAN,AVF 4
2,3-D MAN/CHARLES CHAN,AVF 5
3,3-D MAN/CHARLES CHAN,COC 1
4,3-D MAN/CHARLES CHAN,H2 251


In [211]:
nodes.head()

,node,type
0,2001 10,comic
1,2001 8,comic
2,2001 9,comic
3,24-HOUR MAN/EMMANUEL,hero
4,3-D MAN/CHARLES CHAN,hero


In [309]:
hero = nodes[nodes.type=="hero"]
hero.drop("type", axis=1, inplace=True)


C:\Users\matteo\AppData\Local\Temp\ipykernel_6832\2443088000.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hero.drop("type", axis=1, inplace=True)


In [315]:
#Create empty graph
G2 = nx.Graph()

#Preprocess info for heroes
hero = nodes[nodes.type=="hero"]
hero.drop("type", axis=1, inplace=True)

#Add hero nodes
G2.add_nodes_from(hero.node, prova= "hero")

#Preprocess info for comic
comic = nodes[nodes.type=="comic"]
comic.drop("type", axis=1, inplace=True)

#Add comic nodes
G2.add_nodes_from(comic.node, prova= "comic")

#Preprocess info for edges
a = e.to_numpy()

#Add edges
G2.add_edges_from(a)

C:\Users\matteo\AppData\Local\Temp\ipykernel_6832\2006817898.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hero.drop("type", axis=1, inplace=True)
C:\Users\matteo\AppData\Local\Temp\ipykernel_6832\2006817898.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comic.drop("type", axis=1, inplace=True)


In [ ]:
# TODO: elimina avvisi

## Functionality 1

In [239]:
e.head()

,hero,comic
0,24-HOUR MAN/EMMANUEL,AA2 35
1,3-D MAN/CHARLES CHAN,AVF 4
2,3-D MAN/CHARLES CHAN,AVF 5
3,3-D MAN/CHARLES CHAN,COC 1
4,3-D MAN/CHARLES CHAN,H2 251


In [ ]:
def findNHeroes(edges: pd.DataFrame, N: int) -> pd.DataFrame:
    NHeroes = list(e.groupby(by="hero")
    .count()                                            #count number of repetition
    .sort_values(by="comic", ascending=False)           #Sort values by "comic"
    .head(N)                                            #get top N
    .reset_index()                                      #reset index
    .hero)                                              #get only the names of our heroes
    return NHeroes

In [284]:
N = 20
NHeroes = list(e.groupby(by="hero")
    .count()                                            #count number of repetition
    .sort_values(by="comic", ascending=False)           #Sort values by "comic"
    .tail(N)                                            #get top N
    .reset_index()                                      #reset index
    .hero)   

In [ ]:
def function(graph, typeG, edges, NHeroes = None):

    N = len(graph)

    #If we have the number of top heroes
    if NHeroes != None:
        
        #Then we find the top N Nodes/Heroes
        N = NHeroes
        topN = findNHeroes(edges, N)

        if typeG == "1":
            #take the subset of the graph with all the relatives edges and nodes
            subGraph = graph.edge_subgraph(graph.edges(NHeroes))
            #take the number of nodes
            numberNodes = len(subGraph)
        if typeG == "2":
            print()
            

    return None

In [348]:
ciaociao =dict(G2.nodes(data="prova"))

In [353]:
[k for k, v in ciaociao.items() if v == "comic"]

['BLADE',
 'REBEL',
 'SABRE',
 '2001 10',
 '2001 8',
 '2001 9',
 "A '00",
 "A '01",
 'A 100',
 'A 101',
 'A 102',
 'A 103',
 'A 104',
 'A 105',
 'A 106',
 'A 107',
 'A 108',
 'A 109',
 'A 10',
 'A 110',
 'A 111',
 'A 112',
 'A 113',
 'A 114',
 'A 115',
 'A 116',
 'A 117',
 'A 118',
 'A 119',
 'A 11',
 'A 120',
 'A 121',
 'A 122',
 'A 123',
 'A 124',
 'A 125',
 'A 126',
 'A 127',
 'A 128',
 'A 129',
 'A 12',
 'A 130',
 'A 131',
 'A 132',
 'A 133',
 'A 134',
 'A 135',
 'A 137',
 'A 138',
 'A 139',
 'A 13',
 'A 140',
 'A 141',
 'A 142',
 'A 143',
 'A 144',
 'A 145',
 'A 146',
 'A 147',
 'A 148',
 'A 149',
 'A 14',
 'A 150',
 'A 151',
 'A 152',
 'A 153',
 'A 154',
 'A 155',
 'A 156',
 'A 157',
 'A 158',
 'A 159',
 'A 1.5',
 'A 15',
 'A 160',
 'A 161',
 'A 162',
 'A 163',
 'A 164',
 'A 165',
 'A 166',
 'A 167',
 'A 168',
 'A 169',
 'A 16',
 'A 170',
 'A 171',
 'A 17/2',
 'A 172',
 'A 173',
 'A 174',
 'A 175',
 'A 176',
 'A 177',
 'A 178',
 'A 179',
 'A 17',
 'A 180',
 'A 181',
 'A 18/2',
 '